In [1]:
import os
import glob
import json
import operator

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks
from keras.optimizers import SGD

import coremltools

Using TensorFlow backend.


In [17]:
weights = 'disdat-104-v5-softmax-lr0.001-decay0'
label_file='labels.txt'
data_folder='../clean-data'
output='disdatkerasv7'
num_classes=104

In [3]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [4]:
input_shape=(7,7,512)

In [5]:
datagen = ImageDataGenerator(rescale=1. / 255)
data_generator = datagen.flow_from_directory(
    data_folder,
    target_size=(150, 150),
    batch_size=11,
    class_mode='sparse'
)

class_dictionary = data_generator.class_indices

Found 115042 images belonging to 104 classes.


In [6]:
sorted_predictions = sorted(class_dictionary.items(), key=operator.itemgetter(1))
labels=[p[0] for p in sorted_predictions]

with open(label_file, 'w') as f:
    for label in labels:
        f.write(label+"\n")

In [7]:
with open(weights+'.config') as config_file:    
    config=json.load(config_file)

top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))
decay=1/100 * (config['lr'])/(100000/config['batch_size'])
top_model.compile(optimizer=SGD(lr=config['lr'], decay=config['decay']),
              loss='categorical_crossentropy', metrics=['accuracy'])

top_model.load_weights(weights+'.h5')

In [8]:
model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

Save the model

In [9]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
coreml_model  = coremltools.converters.keras.convert(model,
                                                    image_input_names='image',
                                                    class_labels=label_file,
                                                    input_names=['image'],  
#                                                     red_bias=-123.682/255,
#                                                     green_bias=-116.78/255,
#                                                     blue_bias=-103.94/255,
                                                    image_scale=1/255.0
)

0 : input_1, <keras.engine.topology.InputLayer object at 0x7f93148a59d0>
1 : block1_conv1, <keras.layers.convolutional.Conv2D object at 0x7f92eca44750>
2 : block1_conv1__activation__, <keras.layers.core.Activation object at 0x7f92dd4fafd0>
3 : block1_conv2, <keras.layers.convolutional.Conv2D object at 0x7f92dd896290>
4 : block1_conv2__activation__, <keras.layers.core.Activation object at 0x7f92db9ee1d0>
5 : block1_pool, <keras.layers.pooling.MaxPooling2D object at 0x7f92dd896d90>
6 : block2_conv1, <keras.layers.convolutional.Conv2D object at 0x7f92dd83df50>
7 : block2_conv1__activation__, <keras.layers.core.Activation object at 0x7f92d9f04e90>
8 : block2_conv2, <keras.layers.convolutional.Conv2D object at 0x7f92dd852810>
9 : block2_conv2__activation__, <keras.layers.core.Activation object at 0x7f92dd519190>
10 : block2_pool, <keras.layers.pooling.MaxPooling2D object at 0x7f92dd86ed50>
11 : block3_conv1, <keras.layers.convolutional.Conv2D object at 0x7f92dd815d10>
12 : block3_conv1__act

In [11]:
coreml_model.save(output+'top.mlmodel')

## Full model weights

In [13]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

#model.load_weights(weights+'-refined.h5')
model.load_weights('disdat-104-v5-softmax-lr0.001-decay0-refined-ep12-valacc0.75.h5')
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=2e-4, momentum=0.9),
              metrics=['accuracy'])

In [14]:
coreml_model  = coremltools.converters.keras.convert(model,
                                                    image_input_names='image',
                                                    class_labels=label_file,
                                                    input_names=['image'],  
#                                                     red_bias=-123.68/255,
#                                                     green_bias=-116.78/255,
#                                                     blue_bias=-103.94/255,
                                                    image_scale=1/255.
)

0 : input_3, <keras.engine.topology.InputLayer object at 0x7f92dd165490>
1 : block1_conv1, <keras.layers.convolutional.Conv2D object at 0x7f92dd165550>
2 : block1_conv1__activation__, <keras.layers.core.Activation object at 0x7f92dd1650d0>
3 : block1_conv2, <keras.layers.convolutional.Conv2D object at 0x7f92dd1655d0>
4 : block1_conv2__activation__, <keras.layers.core.Activation object at 0x7f92db9aed50>
5 : block1_pool, <keras.layers.pooling.MaxPooling2D object at 0x7f92dd1658d0>
6 : block2_conv1, <keras.layers.convolutional.Conv2D object at 0x7f92dd15ae50>
7 : block2_conv1__activation__, <keras.layers.core.Activation object at 0x7f92db94a0d0>
8 : block2_conv2, <keras.layers.convolutional.Conv2D object at 0x7f92dd15af90>
9 : block2_conv2__activation__, <keras.layers.core.Activation object at 0x7f92db94a110>
10 : block2_pool, <keras.layers.pooling.MaxPooling2D object at 0x7f92dd0b0d10>
11 : block3_conv1, <keras.layers.convolutional.Conv2D object at 0x7f92dd0e7b10>
12 : block3_conv1__act

In [18]:
coreml_model.save(output+'.mlmodel')

In [16]:
import balloonml

img_path='test_images/'
images = ['calculator.jpg',
'dog.jpg',
'cat.jpg',
'keyboard.jpg',
'macbook.jpg',
'mouse.jpg',
'banana.jpg']
for image in images:
    print('Predicting '+image)
    path = img_path+image
    print(balloonml.predict_top(path, weights+'.h5',class_dictionary))
    print(balloonml.predict_fine(model,class_dictionary, img_path=path))

Predicting calculator.jpg
1/1 [==============================] - 0s
[('calculator', 0.98614025), ('keyboard', 0.005407881), ('hairbrush', 0.0020927957), ('clock', 0.0010452823)]
[('calculator', 0.99983561), ('keyboard', 0.00013622294), ('puzzle', 1.6809556e-05), ('computer mouse', 2.7440112e-06)]
Predicting dog.jpg
1/1 [==============================] - 0s
[('cat', 0.39209151), ('dog', 0.30477524), ('sofa', 0.049107771), ('dinosaur', 0.027725155)]
[('dog', 0.70970178), ('cat', 0.15696621), ('dinosaur', 0.017226743), ('ball', 0.016994534)]
Predicting cat.jpg
1/1 [==============================] - 0s
[('cat', 0.99491477), ('dog', 0.0043442352), ('face', 7.9988407e-05), ('fish', 7.9657184e-05)]
[('cat', 0.99981731), ('box', 0.00015251513), ('star', 1.3727606e-05), ('keyboard', 3.0921935e-06)]
Predicting keyboard.jpg
1/1 [==============================] - 0s
[('blackboard', 0.1212211), ('towel', 0.058512099), ('ruler', 0.049326621), ('shower', 0.04377545)]
[('ruler', 0.32487085), ('keyboar

## Test images for v5

Predicting calculator.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('calculator', 0.021121915), ('keyboard', 0.00038548166), ('key', 6.198656e-05), ('hairbrush', 5.6152505e-05)]
[('calculator', 0.2597464), ('keyboard', 0.00038322349), ('key', 9.7186203e-06), ('computer mouse', 2.592287e-06)]
Predicting dog.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('cat', 0.0047040172), ('dog', 0.0040679271), ('sofa', 0.00085722323), ('dinosaur', 0.00027150256)]
[('cat', 0.0052334545), ('dog', 0.0040698401), ('dinosaur', 0.00085430447), ('sofa', 0.00056111149)]
Predicting cat.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('cat', 0.29206869), ('dog', 0.0055200113), ('star', 5.3297485e-05), ('dinosaur', 3.7203841e-05)]
[('cat', 0.35923439), ('dog', 0.00079333514), ('star', 2.7750855e-05), ('dinosaur', 1.5055543e-05)]
Predicting keyboard.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('radiator', 0.0022044962), ('curtain', 0.00061480148), ('ruler', 0.0005549785), ('blackboard', 0.00050748681)]
[('ruler', 0.0010941911), ('keyboard', 0.00091195788), ('computer mouse', 0.00040513425), ('guitar', 0.00040082214)]
Predicting macbook.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('keyboard', 0.01001082), ('monitor', 0.00050369196), ('blackboard', 0.00025121815), ('computer mouse', 0.00018567205)]
[('keyboard', 0.047883309), ('monitor', 0.00047867949), ('computer mouse', 0.00031897088), ('calculator', 0.00015937244)]
Predicting mouse.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('soap', 0.0042663808), ('computer mouse', 0.00091027975), ('bowl', 0.00067775662), ('pan', 0.00041909842)]
[('soap', 0.0056782723), ('computer mouse', 0.0010733134), ('bowl', 0.00028223582), ('hat', 0.00015233312)]
Predicting banana.jpg
1/1 [==============================] - 0s
1/1 [==============================] - 0s
[('banana', 0.012315591), ('bowl', 0.00019367522), ('sock', 0.00017960947), ('carrot', 5.7873309e-05)]
[('banana', 0.043339983), ('bowl', 0.00051319768), ('onion', 0.00012041288), ('soap', 0.00011019995)]
